In [42]:
import gc
import os

import pandas as pd
import numpy as np
from tqdm import tqdm

In [3]:
! gzip -dkr data/ * / *.gz

In [4]:
data = pd.read_csv('data/BNBUSDT/BNBUSDT_2019_04_439.csv')
data

,exchange,symbol,timestamp,local_timestamp,id,side,price,amount
0,binance,BNBUSDT,1554076801165000,1554076801286549,23000667,sell,17.5114,3.65
1,binance,BNBUSDT,1554076801165000,1554076801286929,23000668,sell,17.5112,6.63
2,binance,BNBUSDT,1554076801673000,1554076801798547,23000669,buy,17.5352,3.65
3,binance,BNBUSDT,1554076801673000,1554076801800498,23000670,buy,17.5354,21.22
4,binance,BNBUSDT,1554076801680000,1554076801801013,23000671,sell,17.5112,1.35
...,...,...,...,...,...,...,...,...
56263,binance,BNBUSDT,1554163199254000,1554163199374936,23056932,sell,17.8841,178.51
56264,binance,BNBUSDT,1554163199260000,1554163199376429,23056933,sell,17.8826,1.50
56265,binance,BNBUSDT,1554163199514000,1554163199630837,23056934,buy,17.8886,5.01
56266,binance,BNBUSDT,1554163199542000,1554163199659047,23056935,sell,17.8888,251.89


In [5]:
data.columns

Index(['exchange', 'symbol', 'timestamp', 'local_timestamp', 'id', 'side',
       'price', 'amount'],
      dtype='object')

In [29]:
# create time series, combine n rows into 1
def merge_rows(df, n):
    new_rows = []
    for i in range(0, len(df), n):
        new_row = df.iloc[i:i + n].values.flatten().tolist()
        if len(df) > i + n:
            new_row += df.iloc[i + n].values.tolist()
        new_rows.append(new_row)
    new_df = pd.DataFrame(new_rows)
    new_columns = [f'{col}_{i}' for i in range(1, n + 1) for col in df.columns]
    new_columns += [f'{col}_target' for col in df.columns]
    new_df.columns = new_columns

    exchange = new_df['exchange_1']
    symbol = new_df['symbol_1']
    start_timestamp = new_df['timestamp_1']
    end_timestamp = new_df['timestamp_target']
    start_local_timestamp = new_df['local_timestamp_1']
    end_local_timestamp = new_df['local_timestamp_target']

    to_remove = ['exchange', 'symbol', 'timestamp', 'local_timestamp', 'id']
    to_remove_cols = [f'{col}_{i}' for i in range(1, n + 1) for col in to_remove] + ['exchange_target', 'symbol_target',
                                                                                     'id_target']
    new_df = new_df.drop(columns=to_remove_cols)

    new_df['exchange'] = exchange
    new_df['symbol'] = symbol
    new_df['start_timestamp'] = start_timestamp
    new_df['start_local_timestamp'] = start_local_timestamp
    return new_df[['exchange', 'symbol', 'start_timestamp', 'start_local_timestamp'] + list(new_df.columns[:-4])]

In [30]:
merge_rows(data, 16)

,exchange,symbol,start_timestamp,start_local_timestamp,side_1,price_1,amount_1,side_2,price_2,amount_2,...,price_15,amount_15,side_16,price_16,amount_16,timestamp_target,local_timestamp_target,side_target,price_target,amount_target
0,binance,BNBUSDT,1554076801165000,1554076801286549,sell,17.5114,3.65,sell,17.5112,6.63,...,17.5116,0.92,sell,17.5115,37.75,1.554077e+15,1.554077e+15,buy,17.5351,1.15
1,binance,BNBUSDT,1554076806784000,1554076806904014,buy,17.5351,1.15,sell,17.5139,4.17,...,17.5096,6.85,sell,17.5003,17.19,1.554077e+15,1.554077e+15,sell,17.4989,1.00
2,binance,BNBUSDT,1554076811499000,1554076811622314,sell,17.4989,1.00,sell,17.4971,1.36,...,17.4761,28.56,buy,17.4844,145.00,1.554077e+15,1.554077e+15,buy,17.5201,1.96
3,binance,BNBUSDT,1554076814934000,1554076815052049,buy,17.5201,1.96,buy,17.5201,2.41,...,17.5013,10.00,sell,17.5002,7.99,1.554077e+15,1.554077e+15,sell,17.5001,2.01
4,binance,BNBUSDT,1554076832691000,1554076832810866,sell,17.5001,2.01,buy,17.5080,11.38,...,17.5282,0.60,buy,17.5283,1.77,1.554077e+15,1.554077e+15,sell,17.5356,0.60
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3512,binance,BNBUSDT,1554162950167000,1554162950286769,sell,17.9066,0.59,buy,17.9065,0.59,...,17.9002,2.71,buy,17.9004,1.75,1.554163e+15,1.554163e+15,buy,17.9083,3.37
3513,binance,BNBUSDT,1554163000257000,1554163000376978,buy,17.9083,3.37,buy,17.9107,1.50,...,17.9200,28.00,buy,17.9213,20.00,1.554163e+15,1.554163e+15,buy,17.9218,20.29
3514,binance,BNBUSDT,1554163064921000,1554163065045307,buy,17.9218,20.29,buy,17.9230,10.00,...,17.9137,415.32,sell,17.9100,30.00,1.554163e+15,1.554163e+15,buy,17.9093,0.56
3515,binance,BNBUSDT,1554163102152000,1554163102279142,buy,17.9093,0.56,sell,17.9002,2.66,...,17.9000,24.00,sell,17.9000,1.39,1.554163e+15,1.554163e+15,sell,17.9000,1.46


In [33]:
lag = 16
dirs = os.listdir('data')

In [36]:
processed = ['BTCUSDT']

In [40]:
total_files = sum([len(files) for r, d, files in os.walk('data')])
total_files

2890

In [41]:
failed = [('ETH-USDT', 'ETH-USDT_2020_01_1735.csv')]

In [ ]:
f

In [43]:
for sym_dir in dirs:
    if sym_dir in processed or sym_dir.startswith('.'):
        continue

    print(f'Processing {len(processed)} ({(len(processed)) / total_files:.2f}%) / {total_files}:', sym_dir)
    for file in tqdm(os.listdir(f'data/{sym_dir}')):
        if file.endswith('.csv'):
            try:
                data = pd.read_csv(f'data/{sym_dir}/{file}')
                data = merge_rows(data, lag)
                data.to_csv(f'data/{sym_dir}/{file}', index=False)

                del data

                gc.collect()
            except Exception as e:
                print(f'Error processing {file}:', e)
                failed.append((sym_dir, file))
    processed.append(sym_dir)

Processing 7 (0.00%) / 2890: SOLBTC


  2%|▏         | 2/103 [00:00<00:05, 19.76it/s]

Error processing SOLBTC_2021_04_561.csv: 'timestamp_1'
Error processing SOLBTC_2023_07_588.csv: 'timestamp_1'
Error processing SOLBTC_2022_08_2744.csv: 'timestamp_1'
Error processing SOLBTC_2021_12_849.csv: 'timestamp_1'
Error processing SOLBTC_2023_01_862.csv: 'timestamp_1'
Error processing SOLBTC_2022_09_2745.csv: 'timestamp_1'
Error processing SOLBTC_2023_09_870.csv: Length mismatch: Expected axis has 513 elements, new values have 969 elements


  8%|▊         | 8/103 [00:00<00:02, 39.81it/s]

Error processing SOLBTC_2023_10_871.csv: 'timestamp_1'
Error processing SOLBTC_2022_02_851.csv: 'timestamp_1'


...

 99%|█████████▉| 87/88 [03:43<00:00,  1.00it/s]

Error processing BNBUSDT_2021_12_15.csv: No columns to parse from file


100%|██████████| 88/88 [03:44<00:00,  2.55s/it]


Processing 41 (0.01%) / 2890: LTCUSDT


100%|██████████| 131/131 [02:31<00:00,  1.15s/it]


In [46]:
len(failed) / total_files

0.03771626297577855

In [47]:
failed

[('ETH-USDT', 'ETH-USDT_2020_01_1735.csv'),
 ('SOLBTC', 'SOLBTC_2021_04_561.csv'),
 ('SOLBTC', 'SOLBTC_2023_07_588.csv'),
 ('SOLBTC', 'SOLBTC_2022_08_2744.csv'),
 ('SOLBTC', 'SOLBTC_2021_12_849.csv'),
 ('SOLBTC', 'SOLBTC_2023_01_862.csv'),
 ('SOLBTC', 'SOLBTC_2022_09_2745.csv'),
 ('SOLBTC', 'SOLBTC_2023_09_870.csv'),
 ('SOLBTC', 'SOLBTC_2023_10_871.csv'),
 ('SOLBTC', 'SOLBTC_2022_02_851.csv'),
 ('SOLBTC', 'SOLBTC_2023_12_593.csv'),
 ('SOLBTC', 'SOLBTC_2023_12_2760.csv'),
 ('SOLBTC', 'SOLBTC_2022_08_577.csv'),
 ('SOLBTC', 'SOLBTC_2021_05_562.csv'),
 ('SOLBTC', 'SOLBTC_2023_02_583.csv'),
 ('SOLBTC', 'SOLBTC_2021_02_2726.csv'),
 ('SOLBTC', 'SOLBTC_2022_01_570.csv'),
 ('SOLBTC', 'SOLBTC_2021_03_2727.csv'),
 ('SOLBTC', 'SOLBTC_2021_08_2732.csv'),
 ('SOLBTC', 'SOLBTC_2023_11_872.csv'),
 ('SOLBTC', 'SOLBTC_2023_03_584.csv'),
 ('SOLBTC', 'SOLBTC_2022_03_852.csv'),
 ('SOLBTC', 'SOLBTC_2021_11_568.csv'),
 ('SOLBTC', 'SOLBTC_2021_09_2733.csv'),
 ('SOLBTC', 'SOLBTC_2022_01_850.csv'),
 ('SOLBTC', '

In [51]:
columns = [
    'exchange', 'symbol', 'start_timestamp', 'start_local_timestamp',
    'side_1', 'price_1', 'amount_1', 'side_2', 'price_2', 'amount_2',
    'side_3', 'price_3', 'amount_3', 'side_4', 'price_4', 'amount_4',
    'side_5', 'price_5', 'amount_5', 'side_6', 'price_6', 'amount_6',
    'side_7', 'price_7', 'amount_7', 'side_8', 'price_8', 'amount_8',
    'side_9', 'price_9', 'amount_9', 'side_10', 'price_10', 'amount_10',
    'side_11', 'price_11', 'amount_11', 'side_12', 'price_12', 'amount_12',
    'side_13', 'price_13', 'amount_13', 'side_14', 'price_14', 'amount_14',
    'side_15', 'price_15', 'amount_15', 'side_16', 'price_16', 'amount_16',
    'timestamp_target', 'local_timestamp_target', 'side_target',
    'price_target', 'amount_target'
]

In [57]:
failed_merge = []
processed_merge = []

In [58]:
# merge all of the transformed files into one
dfs = []
for sym_dir in dirs:
    if sym_dir.startswith('.'):
        continue

    for file in tqdm(os.listdir(f'data/{sym_dir}')):
        if file.endswith('.csv') and (sym_dir, file) not in processed_merge:
            try:
                data = pd.read_csv(f'data/{sym_dir}/{file}')
                if list(data.columns) != columns:
                    print(f'Error processing {file}: columns do not match')
                    failed_merge.append((sym_dir, file))
                    continue
                dfs.append(data)

                del data
                gc.collect()

                processed_merge.append((sym_dir, file))
            except Exception as e:
                print(f'Error processing {file}:', e)
                failed_merge.append((sym_dir, file))

100%|██████████| 55/55 [00:06<00:00,  9.01it/s]
0it [00:00, ?it/s]
 36%|███▌      | 38/105 [00:05<00:09,  7.38it/s]

Error processing ETH-USDT_2020_01_1735.csv: columns do not match


  6%|▌         | 9/154 [00:00<00:12, 11.79it/s]

Error processing LINK-ETH_2019_12_2395.csv: columns do not match


 23%|██▎       | 35/154 [00:03<00:10, 11.71it/s]

...

 92%|█████████▏| 81/88 [00:13<00:00,  9.46it/s]

Error processing BNBUSDT_2023_11_38.csv: columns do not match


 99%|█████████▉| 87/88 [00:13<00:00,  9.54it/s]

Error processing BNBUSDT_2021_12_15.csv: No columns to parse from file


100%|██████████| 131/131 [00:19<00:00,  6.80it/s]


In [59]:
df = pd.concat(dfs, axis=0, ignore_index=True)
df

,exchange,symbol,start_timestamp,start_local_timestamp,side_1,price_1,amount_1,side_2,price_2,amount_2,...,price_15,amount_15,side_16,price_16,amount_16,timestamp_target,local_timestamp_target,side_target,price_target,amount_target
0,huobi,BTCUSDT,1638316800024000,1638316800030727,buy,56951.09,0.002000,sell,56954.89,0.001755,...,56956.33,0.000344,sell,56956.33,0.003988,1.638317e+15,1.638317e+15,sell,56956.33,0.019312
1,huobi,BTCUSDT,1638316801594000,1638316801600716,sell,56956.33,0.019312,buy,56956.34,0.001782,...,56956.36,0.062500,buy,56956.36,0.000200,1.638317e+15,1.638317e+15,buy,56959.36,0.004289
2,huobi,BTCUSDT,1638316803722000,1638316803728136,buy,56959.36,0.004289,buy,56959.36,0.049821,...,56959.36,0.000911,sell,56959.36,0.000344,1.638317e+15,1.638317e+15,sell,56959.36,0.000149
3,huobi,BTCUSDT,1638316807694000,1638316807700044,sell,56959.36,0.000149,buy,56959.37,0.001304,...,56962.06,0.000200,buy,56962.29,0.003393,1.638317e+15,1.638317e+15,buy,56964.63,0.000200
4,huobi,BTCUSDT,1638316809485000,1638316809509760,buy,56964.63,0.000200,sell,56963.42,0.001113,...,56974.16,0.002000,buy,56966.16,0.095191,1.638317e+15,1.638317e+15,buy,56974.70,0.050476
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21930709,huobi,LTCUSDT,1619913542128000,1619913542147659,buy,276.59,1.702400,buy,276.59,1.553700,...,276.64,0.255000,buy,276.64,0.035100,1.619914e+15,1.619914e+15,buy,276.61,0.024400
21930710,huobi,LTCUSDT,1619913558052000,1619913558062612,buy,276.61,0.024400,sell,276.60,4.000000,...,276.63,0.076800,buy,276.64,0.048800,1.619914e+15,1.619914e+15,buy,276.67,0.080300
21930711,huobi,LTCUSDT,1619913570057000,1619913570064237,buy,276.67,0.080300,buy,276.64,0.001200,...,276.59,0.079300,buy,276.63,2.980900,1.619914e+15,1.619914e+15,buy,276.59,0.086700
21930712,huobi,LTCUSDT,1619913580946000,1619913580972180,buy,276.59,0.086700,buy,276.63,3.555200,...,276.70,0.061200,buy,276.71,5.216500,1.619914e+15,1.619914e+15,buy,276.72,2.489600


In [60]:
df.to_csv('data/final.csv', index=False)

In [61]:
del df

In [9]:
# zip only .gz files
# ! zip -r trades data -i '*.gz'

  adding: data/BTCUSDT/BTCUSDT_2021_08_256.csv.gz (deflated 0%)
  adding: data/BTCUSDT/BTCUSDT_2021_07_2512.csv.gz (deflated 0%)
  adding: data/BTCUSDT/BTCUSDT_2019_12_2493.csv.gz (deflated 0%)
  adding: data/BTCUSDT/BTCUSDT_2020_01_2494.csv.gz (deflated 0%)
  adding: data/BTCUSDT/BTCUSDT_2021_11_2516.csv.gz (deflated 0%)
  adding: data/BTCUSDT/BTCUSDT_2023_04_276.csv.gz (deflated 0%)
  adding: data/BTCUSDT/BTCUSDT_2020_05_241.csv.gz (deflated 0%)
  adding: data/BTCUSDT/BTCUSDT_2023_10_2539.csv.gz (deflated 9%)
  adding: data/BTCUSDT/BTCUSDT_2021_03_251.csv.gz (deflated 0%)
  adding: data/BTCUSDT/BTCUSDT_2021_01_2506.csv.gz (deflated 0%)
  adding: data/BTCUSDT/BTCUSDT_2021_06_2511.csv.gz (deflated 0%)
  adding: data/BTCUSDT/BTCUSDT_2019_09_233.csv.gz (deflated 0%)
  adding: data/BTCUSDT/BTCUSDT_2020_02_238.csv.gz (deflated 0%)
  adding: data/BTCUSDT/BTCUSDT_2021_01_249.csv.gz (deflated 0%)
  adding: data/BTCUSDT/BTCUSDT_2021_10_2515.csv.gz (deflated 0%)
  adding: data/BTCUSDT/BTCUSDT_2

In [10]:
# ! rm -rf data/*/*.gz